# Model Editor

This notebook demonstrates functionality of `yaml2sbmls` model editor using the Lotka Volterra Equations as example. The "Lotka-Volterra" Equations are given by 

\begin{align*}
\frac{d}{dt} x_1 &= \alpha x_1 - \beta x_1x_2, \\
\frac{d}{dt} x_2 &= \delta x_1x_2 - \gamma x_2.
\end{align*}

## ODE model

We first generate a basic model, that only contains all necessary information for generating an SBML file for model simulation.

In [1]:
from yaml2sbml import YamlModel

# generate model
model = YamlModel()

# add ODEs
model.add_ode(state_id='x_1', 
              right_hand_side='alpha*x_1 - beta*x_1*x_2', 
              initial_value=2)
model.add_ode(state_id='x_2', 
              right_hand_side='delta*x_1*x_2 - gamma*x_2', 
              initial_value=2)

# add parameters
model.add_parameter(parameter_id='alpha', nominal_value=2)
model.add_parameter(parameter_id='beta', nominal_value=4)
model.add_parameter(parameter_id='gamma', nominal_value=3)
model.add_parameter(parameter_id='delta', nominal_value=3)

`yaml2sbml` allows to export the `model` object either to YAML or to SBML directly, via  

In [2]:
# write to YAML
model.write_to_yaml('Lotka_Volterra_basic.yaml', over_write=True)

# write to SBML
model.write_to_sbml('Lotka_Volterra_basic.xml', over_write=True)

There are further functions to: 
* get all `parameter_id`s via `model.get_parameter_ids()` 
* get a parameter by its id (`model.get_parameter_by_id('alpha')`) 
* delete a parameter by its id (`model.delete_parameter('alpha')`)

Similar functions also exist for the other model components.

## Parameter Estimation Problem

Now we want to extend the current `model` to include all the necessary information for parameter estimation in PEtab. Therefore we load the model from the `.yaml` file and modify the parameters, such that it also contains all information that is going to be written into the PEtab parameter table.

In [3]:
model = YamlModel.load_from_yaml('Lotka_Volterra_basic.yaml')

# extend parameters 
model.add_parameter(parameter_id='alpha',
                    nominal_value=2,
                    parameter_scale='log10',
                    lower_bound=0.1,
                    upper_bound=10,
                    estimate=1, 
                    over_write=True)

model.add_parameter(parameter_id='beta',
                    nominal_value=4,
                    parameter_scale='log10',
                    lower_bound=0.1,
                    upper_bound=10,
                    estimate=1, 
                    over_write=True)

model.add_parameter(parameter_id='gamma',
                    nominal_value=3,
                    parameter_scale='log10',
                    lower_bound=0.1,
                    upper_bound=10,
                    estimate=1,
                    over_write=True)

model.add_parameter(parameter_id='delta',
                    nominal_value=3,
                    parameter_scale='log10',
                    lower_bound=0.1,
                    upper_bound=10,
                    estimate=1,
                    over_write=True)

A parameter fitting problem in PEtab allows for the specification of observables and experimental conditions:

In [4]:
# specify an observable:
model.add_observable(observable_id='prey_measured', 
                     observable_formula='log10(x_1)',
                     noise_formula='noiseParameter1_prey_measured', 
                     noise_distribution='normal',
                     observable_transformation='lin')

# specify trivial condition
model.add_condition(condition_id='condition1', 
                    condition_dict={})

The modified model can either be exported to YAML or PEtab via

In [5]:
# write to YAML
model.write_to_yaml('Lotka_Volterra_PEtab.yaml', over_write=True)

# write to PEtab
model.write_to_petab(output_dir='./Lotka_Volterra_PEtab',
                     model_name='Lotka_Volterra',
                     petab_yaml_name='Lotka_Volterra_problem', 
                     measurement_table_name='measurement.tsv')